<a href="https://colab.research.google.com/github/ramzesssina/NLPrespos/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_3(%D0%9F%D1%83%D0%BD%D0%BA3GPT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Пункт 3 - Создание GPT**

In [ ]:
import numpy as np

In [ ]:
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

In [ ]:
class Neuron:
    def __init__(self, input_size):
        self.w = np.random.randn(input_size) * 0.1
        self.b = 0.0

    def feedforward(self, x):
        self.last_x = x
        self.last_total = np.dot(x, self.w) + self.b
        return sigmoid(self.last_total)

    def train(self, grad_output, lr=0.01):
        grad_total = deriv_sigmoid(self.last_total) * grad_output
        self.w -= lr * grad_total * self.last_x
        self.b -= lr * grad_total

In [ ]:
class Head:
    def __init__(self, vocab_size, embed_dim, hidden_dim, lr=0.01):
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.lr = lr

        self.W_k = np.random.randn(embed_dim, hidden_dim) * 0.1
        self.W_q = np.random.randn(embed_dim, hidden_dim) * 0.1
        self.W_v = np.random.randn(embed_dim, hidden_dim) * 0.1

        self.W_out = np.random.randn(hidden_dim, vocab_size) * 0.1
        self.b_out = np.zeros((vocab_size,))

    @staticmethod
    def softmax(x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

    def forward(self, x):
        self.x = x
        B, T, C = x.shape

        self.k = x @ self.W_k
        self.q = x @ self.W_q
        self.v = x @ self.W_v

        self.wei = np.matmul(self.q, self.k.transpose(0, 2, 1)) / np.sqrt(self.hidden_dim)
        mask = np.tril(np.ones((T, T)))
        self.wei = np.where(mask == 1, self.wei, -1e9)
        self.wei_softmax = self.softmax(self.wei)
        self.out = np.matmul(self.wei_softmax, self.v)

        self.logits = np.matmul(self.out, self.W_out) + self.b_out
        self.probs = self.softmax(self.logits)
        return self.probs

    def train(self, dataset, targets, epochs=1000):
        for epoch in range(epochs):
            probs = self.forward(dataset)
            loss = -np.mean(np.log(probs[np.arange(len(dataset))[:, None], np.arange(dataset.shape[1]), targets]))

            dlogits = probs
            dlogits[np.arange(len(dataset))[:, None], np.arange(dataset.shape[1]), targets] -= 1
            dlogits /= len(dataset) * dataset.shape[1]

            grad_W_out = self.out.reshape(-1, self.hidden_dim).T @ dlogits.reshape(-1, self.vocab_size)
            grad_b_out = np.sum(dlogits, axis=(0, 1))

            self.W_out -= self.lr * grad_W_out
            self.b_out -= self.lr * grad_b_out

            if epoch % 100 == 0:
                print(f"Epoch {epoch}: Loss = {loss:.4f}")

    def evaluate(self, text_ids, idx_to_word, embedding_matrix, steps=5, true_text=None, temperature=1.0):
      generated_text = "".join([idx_to_word[i] for i in text_ids])
      print("Начальный текст:", generated_text)

      for step in range(steps):
          input_ids = np.array(text_ids[-4:])
          embedded = embedding_matrix[input_ids].reshape(1, -1, self.embed_dim)
          probs = self.forward(embedded)[0, -1]

          probs = np.log(probs + 1e-8) / temperature
          probs = np.exp(probs) / np.sum(np.exp(probs))

          next_id = np.random.choice(len(probs), p=probs)
          text_ids.append(next_id)
          predicted_word = idx_to_word[next_id]  # Используем idx_to_word, а не idx_to_char
          if true_text is not None and step + 1 < len(true_text):
              actual_word = true_text[step + 1]
              print(f"Предсказанное слово: {predicted_word}, Реальное слово: {actual_word}")
          else:
              print(f"Предсказанное слово: {predicted_word}")

In [ ]:
text = "Вдохновляясь новыми технологиями, я решительно настроен углубить свои знания в веб-разработке и автоматизации, чтобы быстро достичь профессиональных успехов. Мое стремление к совершенству подкрепляется ежедневными занятиями и практическими проектами."
words = text.split()

word_to_idx = {word: i for i, word in enumerate(sorted(set(words)))}
idx_to_word = {i: word for word, i in word_to_idx.items()}
vocab_size = len(word_to_idx)

In [ ]:
seq_len = 4
X = []
Y = []

for i in range(len(words) - seq_len):
    seq = words[i:i+seq_len]
    target = words[i+1:i+seq_len+1]
    X.append([word_to_idx[word] for word in seq])
    Y.append([word_to_idx[word] for word in target])

X = np.array(X)
Y = np.array(Y)

In [ ]:
embedding_dim = 2
embedding_matrix = np.random.randn(vocab_size, embedding_dim) * 0.1
X_embedded = embedding_matrix[X]

In [ ]:
X = np.array(X)
Y = np.array(Y)

embedding_dim = 2
embedding_matrix = np.random.randn(vocab_size, embedding_dim) * 0.1
X_embedded = embedding_matrix[X]

model.train(X_embedded, Y, epochs=1000)

Epoch 0: Loss = 3.2498
Epoch 100: Loss = 3.2411
Epoch 200: Loss = 3.2361
Epoch 300: Loss = 3.2331
Epoch 400: Loss = 3.2310
Epoch 500: Loss = 3.2295
Epoch 600: Loss = 3.2283
Epoch 700: Loss = 3.2273
Epoch 800: Loss = 3.2265
Epoch 900: Loss = 3.2258


In [ ]:
initial_text = "Вдохновляясь"
text_ids = [word_to_idx[word] for word in initial_text.split()]

true_text = text[len(initial_text):len(initial_text)+5]
model.evaluate(text_ids, idx_to_word, embedding_matrix, true_text=true_text)

Начальный текст: Вдохновляясь
Предсказанное слово: ежедневными, Реальное слово: н
Предсказанное слово: и, Реальное слово: о
Предсказанное слово: свои, Реальное слово: в
Предсказанное слово: ежедневными, Реальное слово: ы
Предсказанное слово: настроен
